In [1]:
%pwd

'/workspaces/Text-Summarization/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/workspaces/Text-Summarization'

## Entities 

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


## configurtions manager in src/config

In [8]:
from textSummarization.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textSummarization.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager():
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        print("Configuration Manager Initiated")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        
        config = self.config.data_transformation

        create_directories([Path(config.root_dir)])

        return DataTransformationConfig(
            root_dir = Path(config.root_dir),
            data_path= Path(config.data_path),
            tokenizer_name= Path(config.tokenizer_name)
        )
            

## Components

In [10]:
import os
from textSummarization.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(
            example_batch['dialogue'],
            max_length=1024,
            truncation=True
        )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                example_batch['summary'],
                max_length=128,
                truncation=True
                )
            
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)

        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_features,
            batched=True
        )

        dataset_samsum_pt.save_to_disk(
            os.path.join(self.config.root_dir, 'samsum_dataset')
            )


## Pipeline

In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e


Configuration Manager Initiated
[2023-10-09 07:56:12,577: INFO: common] yaml file: config/config.yaml loaded successfully
[2023-10-09 07:56:12,591: INFO: common] yaml file: params.yaml loaded successfully
[2023-10-09 07:56:12,592: INFO: common] directory: artifacts created successfully
[2023-10-09 07:56:12,593: INFO: common] directory: artifacts/data_transformation created successfully


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/workspaces/Text-Summarization/env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3864: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 175810.44 examples/s]
